In [12]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])\

import sys
sys.path.append('../..')
import utils

debug = True

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [13]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

In [14]:
print(utils.get_opp_by_name("Tech for Good"))

{'nonprofit_organization': 'Tech for Good', 'location': 'Remote', 'category': 'Technology Education', 'volunteering_days': ['Flexible'], 'description': 'Share your tech expertise remotely by mentoring students and teaching them valuable digital skills and programming.', 'group_size': '1-1', 'start_date': '2023-07-15', 'end_date': '2024-06-30', 'desired_skills': ['Programming', 'Communication'], 'volunteer_type': 'Virtual'}


In [15]:
print(utils.get_opps_by_category("Animal Shelter"))

[{'nonprofit_organization': 'Animal Guardians', 'location': 'Los Angeles', 'category': 'Animal Shelter', 'volunteering_days': ['Monday', 'Tuesday', 'Friday'], 'description': 'Assist in caring for shelter animals by providing feeding, grooming, and exercise, and help find them forever homes.', 'group_size': '1-5', 'start_date': '2023-07-01', 'end_date': '2023-09-30', 'desired_skills': ['Animal Handling', 'Compassion'], 'volunteer_type': 'In-person'}]


In [16]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"

    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    moderation_flagged = check_moderation_flag(user_input)
    if moderation_flagged:
        if debug: print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."
    
    if debug: print("Step 1: Input passed moderation check.\n")
    
    # Step 2: Extract the list of opportunities
    if debug: print("Step 2: Extract list of opportunities.")
    category_and_opp_response = extract_opportunities(user_input, debug)
    category_and_opp_list = utils.read_string_to_list(category_and_opp_response)
    if debug: 
        print("category_and_opp_list\n")
        print(category_and_opp_list)

    # Step 3: Look up opportunities information
    if debug: print("Step 3: Look up opportunities information.\n")
    opp_information = lookup_opportunities(category_and_opp_list, debug)

    # Step 4: Generate response to user question
    if debug: print("Step 4: Generate response to user question.\n")
    messages = generate_response_messages(user_input, opp_information, debug)

    # Step 5: Get the final response
    final_response = get_completion_from_messages(all_messages + messages)
    moderation_passed = check_moderation_flag(final_response)
    if moderation_passed:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    # Step 6: Model evaluates the response
    print("Step 6: Use model to evaluate the response.\n")
    evaluation_response = evaluate_response(user_input, final_response, debug)

    # Step 7: Decide whether to use the answer or connect to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.\n")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.\n")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a person for further assistance."
        return neg_str, all_messages

def check_moderation_flag(input_text):
    response = client.moderations.create(input=input_text)
    response_dict = response.model_dump()
    is_flagged = response_dict['results'][0]['flagged']
    return is_flagged

def extract_opportunities(user_input, debug=True):
    categories_and_opportunities=utils.get_opps_and_category()
    if debug:
        print("Categories:\n")
        for category in categories_and_opportunities.keys():
            print(category)
        print("Categories and Opportunities:\n")
        print(categories_and_opportunities)
    category_and_opp_response = utils.find_category_and_opp_only(user_input)
    if debug: 
        print("category_and_opp_response\n")
        print(category_and_opp_response)
        print()
    return category_and_opp_response

def lookup_opportunities(category_and_opp_list, debug=True):
    opp_information = utils.generate_output_string(category_and_opp_list)
    if debug: 
        print(opp_information)
    return opp_information

def generate_response_messages(user_input, opp_information, debug=True):
    delimiter = "```"
    system_message = """
    You are a customer service assistant that helps people find volunteer opportunities at non-profits. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant opportunities information:\n{opp_information}"}
    ]
    if debug:
        for message in messages:
            print(message)
            print()
    return messages

def evaluate_response(user_input, final_response, debug=True):
    delimiter = "```"
    system_message = """
    You are a customer service assistant that helps people find volunteer opportunities at non-profits. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug:
        for message in messages:
            print(message)
            print()
        print(evaluation_response)
        print()
    return evaluation_response

# Example usage
user_input = "tell me about opportunities at a Homeless Shelter"
response, _ = process_user_message(user_input, [], debug=True)

print(response)

Step 1: Input passed moderation check.

Step 2: Extract list of opportunities.
Categories:

Homeless Shelter
Environmental Cleanup
Youth Mentoring
Animal Shelter
Food Bank
Community Development
Education
Technology Education
Mental Health
Language Access
Community Engagement
Urban Gardening
Arts Education
Elderly Care
Youth Sports
Categories and Opportunities:

{'Homeless Shelter': ['Helping Hands Foundation'], 'Environmental Cleanup': ['Eco Warriors'], 'Youth Mentoring': ['Youth Empowerment League'], 'Animal Shelter': ['Animal Guardians'], 'Food Bank': ['Food for All'], 'Community Development': ['Community Builders'], 'Education': ['Virtual Tutoring Program'], 'Technology Education': ['Tech for Good'], 'Mental Health': ['Youth Helpline'], 'Language Access': ['Remote Translation Services'], 'Community Engagement': ['Online Community Support'], 'Urban Gardening': ['Green Thumb Society'], 'Arts Education': ['Creative Arts Center'], 'Elderly Care': ['Senior Companions'], 'Youth Sports': [

In [17]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_opps_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [19]:
debug = False
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)